<div style="background-color: lightgreen; border-radius: 5px; padding: 10px;">
    <h4>Topic Modeling</h4>
    <p>...</p>
</div>

In [85]:
import os
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm.notebook import tqdm

from preprocessing_functions import *
from topic_modeling_functions import *

import warnings
warnings.filterwarnings('ignore')

# Saving and visualising lda models
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Variables

In [68]:
files_dir = '../../../../data_clean/paragraphs_lemmatised/'
POS = ["NOUN", "VERB", "ADJ"]




ONLY_ENGLISH_WORDS = True
merged_POS = True
saving = True

# Don't change!
word_type = 'english_words' if ONLY_ENGLISH_WORDS else 'dirty_words'
files_output_dir = f"{files_dir}/{word_type}_merged_{''.join(POS)}"

In [47]:
os.makedirs(f"{files_dir}/merged_{''.join(POS)}", exist_ok=True)

### Merge POS (Should be removed to pre-processing notebook!)

In [72]:
# for file in os.listdir()
for file in os.listdir(files_dir)[:]:
    if file.endswith(".csv"):
        
        output_fp = os.path.join(files_output_dir, file.replace('preprocessed', 'merged_POS'))
        
        if os.path.isfile(output_fp):
            print(f"{output_fp} already exists.")
            continue
            
        fp = os.path.join(files_dir, file)
        temp_df = pd.read_csv(fp)

        if merged_POS:
            os.makedirs(files_output_dir, exist_ok=True)
            temp_df['merged_POS'] = [[] for _ in range(temp_df.shape[0])]

        for tag in POS:
            if ONLY_ENGLISH_WORDS:
                        column_name = f'{tag}_clean'   
            else:
                column_name = f'{tag}'

            string_to_list = temp_df[column_name].apply(literal_eval)

            if merged_POS:
                temp_df['merged_POS'] += string_to_list
        
        # Save df
        temp_df = temp_df[['city_1', 'city_2', 'paragraph_id', 'paragraph', 'article_id', 'title', 'merged_POS']]
        temp_df.to_csv(output_fp, index=False)

../../../../data_clean/paragraphs_lemmatised//english_words_merged_NOUNVERBADJ\paragraphs_10_934384_1038204_merged_POS.csv already exists.
../../../../data_clean/paragraphs_lemmatised//english_words_merged_NOUNVERBADJ\paragraphs_11_1038204_1142024_merged_POS.csv already exists.
../../../../data_clean/paragraphs_lemmatised//english_words_merged_NOUNVERBADJ\paragraphs_12_1142024_1245844_merged_POS.csv already exists.


In [73]:
temp_df[['city_1', 'city_2', 'paragraph_id', 'paragraph', 'article_id', 'title', 'merged_POS']].head(5)

,city_1,city_2,paragraph_id,paragraph,article_id,title,merged_POS
0,Paris,London,830565,Piip was a member of the Estonian Province Ass...,11414959,Ants Piip,"[member, member, member, delegation, peace, ne..."
1,London,Paris,830566,Piip was a member of the Estonian Province Ass...,11414959,Ants Piip,"[member, member, member, delegation, peace, ne..."
2,Paris,Munich,830567,"Humphrey was born in Saint John, New Brunswick...",11415024,Jack Humphrey,"[school, student, school, bear, study, travel,..."
3,Munich,Paris,830568,"Humphrey was born in Saint John, New Brunswick...",11415024,Jack Humphrey,"[school, student, school, bear, study, travel,..."
4,Turin,Bologna,830569,"Born in Bologna, at the age of 15, he began tr...",11415085,Giuseppe Pedretti,"[age, opposition, family, son, studio, bear, b..."


In [75]:
dataframes = []

for file in tqdm(os.listdir(files_output_dir)):
    fp = os.path.join(files_output_dir, file)
    
    temp_df = pd.read_csv(fp)
    dataframes.append(temp_df)

In [81]:
len(dataframes)

20

In [82]:
# frames = [citypair['lemmatized_paragraphs'] for citypair in data_list]
# citypairs = [citypair['city_pair'] for citypair in data_list]

result = pd.concat(dataframes) #, keys=citypairs)
result.set_index('paragraph_id', inplace=True)
result.sort_index(inplace=True)
# result.reset_index(inplace=True)

# paris_london = result.loc["paris_london"]
# paris_london.set_index('paragraph_id', inplace=True)
# paris_london

result #.iloc[21051]

,city_1,city_2,paragraph,article_id,title,merged_POS
paragraph_id,,,,,,
1,Birmingham,Florence,The first community of adherents of the Baha'i...,303,Alabama,"['community', 'adherent', 'center', 'found', '..."
2,Florence,Birmingham,The first community of adherents of the Baha'i...,303,Alabama,"['community', 'adherent', 'center', 'found', '..."
3,Paris,London,A major revision of the work by composer and a...,309,An American in Paris,"['revision', 'work', 'composer', 'arranger', '..."
4,London,Paris,A major revision of the work by composer and a...,309,An American in Paris,"['revision', 'work', 'composer', 'arranger', '..."
5,Madrid,Rome,Access to biocapacity in Algeria is lower than...,358,Algeria,"['access', 'world', 'hectare', 'person', 'terr..."
...,...,...,...,...,...,...
2076400,Paris,Stockholm,Mundhir participated in Arab and international...,70585176,Mundhir Masri,"['festival', 'seminar', 'guest', 'participate'..."
2076401,London,Paris,Mundhir participated in Arab and international...,70585176,Mundhir Masri,"['festival', 'seminar', 'guest', 'participate'..."
2076402,London,Stockholm,Mundhir participated in Arab and international...,70585176,Mundhir Masri,"['festival', 'seminar', 'guest', 'participate'..."


## Vectorize data

In [108]:
%%time

result.merged_POS =  result.merged_POS.apply(literal_eval)

CPU times: total: 10min 52s
Wall time: 13min 51s


In [ ]:
%%time

paragraphs = result.merged_POS
MIN_DF = 0.1
MAX_DF = 0.8

# Vectorization
dictionary, corpus = vectorize(paragraphs, MIN_DF=MIN_DF, MAX_DF=MAX_DF)

## Train Single LDA Model

In [ ]:
N_TOPICS = # e.g. 6
model_dict = train_lda_model(lemmatized_text=paragraphs,
                            dictionary=dictionary,
                            corpus=corpus,
                            MIN_DF=MIN_DF,
                            MAX_DF=MAX_DF,
                            N_TOPICS=N_TOPICS
                            # random_seed=0, should be added to allow the same results!
                            )

print(model_dict.keys())

## Load Single LDA Model

### Only run if a). you want to load a previously trained model or b). the model_dict variable is empty

In [ ]:
# Give path to model
single_model_path = #
forced_single_model_path = os.path.abspath(single_model_path)

# model = load_lda_model(forced_single_model_path, LOAD_VIS=True, LOAD_DICT=True, LOAD_TEXTS=True, LOAD_COHERENCE_SCORE=True)

LOAD_VIS=True,
LOAD_DICT=True,
LOAD_TEXTS=True,
LOAD_COHERENCE_SCORE=True

if os.path.exists(forced_single_model_path):
    model_dict = load_lda_model(forced_single_model_path,
                           LOAD_VIS=LOAD_VIS,
                           LOAD_DICT=LOAD_DICT,
                           LOAD_TEXTS=LOAD_TEXTS,
                           LOAD_COHERENCE_SCORE=LOAD_COHERENCE_SCORE)
    print(model_dict.keys())
else:
    print('Path is invalid')

## Visualisation

In [ ]:
def visualise_topics(lda_model, corpus, dictionary, sort_topics=False):
    lda_conv = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_model)
    vis = gensimvis.prepare(lda_conv, corpus, dictionary, sort_topics=False)
    
    print(vis.topic_order)
    
    return vis

In [ ]:
pyLDAvis.enable_notebook()

if 'visualisation' not in model_dict.keys():
    vis = visualise_topics(model_dict['lda_model'], corpus, dictionary)
else:
    vis = model_dict['visualisation']

vis

## Get most relevant words per topic

#### (used to create the topic vectors in the word embedding classification method)

In [ ]:
all_topics = {}
num_terms = 15 # Adjust number of words to represent each topic
lambd = 0.2 # Adjust this accordingly based on tuning above

if not isinstance(vis, pyLDAvis._prepare.PreparedData):
    vis = visualise_topics(model_dict['lda_model'], corpus, dictionary)
    
for i in range(1,len(model_dict['lda_model'].get_topics())+1): #Adjust this to reflect number of topics chosen for final LDA model
    topic = vis.topic_info[vis.topic_info.Category == 'Topic'+str(i)].copy()
    topic['relevance'] = topic['loglift']*(1-lambd)+topic['logprob']*lambd
    all_topics['Topic '+str(i)] = topic.sort_values(by='relevance', ascending=False).Term[:num_terms].values

In [ ]:
pd.DataFrame(all_topics).T
# display(all_topics)

## Word Topic Distribution

In [ ]:
N_TOPICS = len(model_dict['lda_model'].get_topics())
MAX_WORDS = 8

for i in range(N_TOPICS):
    topic_words = model_dict['lda_model'].show_topic(i, topn=MAX_WORDS)
    print(i+1, [(x[0], round(x[1], 3)) for x in topic_words])
    print()

## Document (=paragraphs) topic distribution

In [ ]:
transformed_docs = model_dict['lda_model'].load_document_topics()
topic_distributions = pd.DataFrame([[x[1] for x in doc] for doc in transformed_docs], 
             columns=['topic_{}'.format(i+1) for i in range(N_TOPICS)])

In [ ]:
topic_distributions.head(2)

## Rename Topics

In [ ]:
# Give Topics sensible names
# topic_distributions_renamed_cols=topic_distributions.rename(columns = { 'topic_1': 'lda_sport',
#                                         'topic_2': 'lda_art',
#                                         'topic_3': 'lda_diplomacy',
#                                         'topic_4': 'lda_entertainment',
#                                         'topic_5': 'lda_transportation',
#                                         'topic_6': 'lda_education'}, inplace = False)

topic_distributions_renamed_cols.head(2)

## Topic Distributions

In [ ]:
topic_distributions_renamed_cols.idxmax(axis="columns").value_counts()

In [ ]:
topic_distributions_renamed_cols.idxmax(axis="columns").value_counts(normalize=True)

## Merge paragraphs with topic distributions

In [ ]:
updated_results = pd.concat([result, topic_distributions_renamed_cols],
                  axis = 1)

In [ ]:
updated_results.head(2)

## Get dominant topic and score of chunked dataframe

In [ ]:
nr_of_chunks = (len(updated_results) // 10000) + 1

chunked_dataframe = np.array_split(updated_results, nr_of_chunks)

In [ ]:
# This may take a while

for i, subdataframe in enumerate(tqdm(chunked_dataframe)):
    chunked_dataframe[i] = pd.concat([subdataframe, subdataframe[topic_distributions_renamed_cols.columns].agg(['idxmax','max'],axis=1)], axis=1)

In [ ]:
final_df = pd.concat(chunked_dataframe,axis=0)

In [ ]:
final_df_renamed_cols = final_df.rename(columns = { 'idxmax': 'lda_dominant',
                                        'max': 'lda_dominant_score'}, inplace = False)

In [ ]:
final_df_renamed_cols.head(2)

## Query of 3 city pairs

In [ ]:
# Requires work due to city pair in two columns
# three_city_pairs = final_df_renamed_cols[final_df_renamed_cols['city_pair'].isin(['paris_milan', 'barcelona_manchester', 'warsaw_prague'])]

In [ ]:
# display(three_city_pairs.groupby('city_pair')['lda_dominant'].value_counts())

## Save document topic distribution

In [ ]:
num, div = len(all_paragraphs_df), 20
chunks = [num // div + (1 if x < num % div else 0)  for x in range (div)]
cum_chunks = [0]

for i, x in enumerate(chunks):
    cum_chunks.append(sum(chunks[:i+1]))

In [ ]:
chunks_min_max = list(zip(cum_chunks, cum_chunks[1:]))
chunks_min_max

In [ ]:
output_folder = "../../../../data_clean/paragraphs_lda_topic_distribution"
os.makedirs(output_folder, exist_ok=True)

count = 1
for chunk in tqdm(chunks_min_max):
    sub_df = all_paragraphs_df.iloc[chunk[0]:chunk[1]]
    file_path = f"{output_folder}/paragraphs_{count}_{chunk[0]}_{chunk[1]}_lda_topics.csv"
    sub_df.to_csv(file_path, index=False)
    count += 1
    
# final_df_renamed_cols.to_csv('..\..\..\..\..\data\clean\lda_classified_30cities_435citypairs_311k_paragraphs.csv', index=False)

## Distribution of dominant topic scores

In [ ]:
final_df_renamed_cols['lda_dominant_score'].value_counts(bins=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]).sort_values(ascending=False)

## Graphed density of the grouped topic scores

In [ ]:
# !pip install seaborn
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns

plt.figure(figsize=(20,10))

a = final_df_renamed_cols.groupby('lda_dominant')

# Iterate through the five airlines
for topic in final_df_renamed_cols['lda_dominant'].unique():
    b = a['lda_dominant_score'].get_group(topic)
    
    #     b.plot.density(color='green')
    # plt.title('Density plot for Speeding')
    # plt.show()
    
    sns.distplot(b, hist = False, kde = True,
                 kde_kws = {'linewidth': 2},
                 label = topic)
    
# Plot formatting
sns.distplot(final_df_renamed_cols['lda_dominant_score'], hist = False, kde = True,
                 kde_kws = {'linewidth': 4},
                 color = 'black',
                 label = 'Average')

plt.legend(prop={'size': 20}, title = 'Topic', title_fontsize=22)
# plt.title('Score Density of the dominant topics grouped by topic', fontsize=24)
plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=24)
plt.rc('xtick', labelsize=24)
plt.rc('ytick', labelsize=24)
plt.xlabel('Topic Dominance')
plt.ylim(0, 12)
plt.xlim(0.2, 1)
plt.ylabel('Density')

# EXTRAS

## Training Multiple LDA Models

In [ ]:
TOPIC_SELECTION = range(2,21, 1)
list(TOPIC_SELECTION)


%%time

paragraphs = result.merged_POS
# MIN_DF = 0.1
# MAX_DF = 0.8

# Vectorization
# dictionary, corpus = vectorize(paragraphs, MIN_DF=MIN_DF, MAX_DF=MAX_DF)

models = compare_lda_models(OUTPUT_DIR='../../../../../data/clean/lda_models/50k_paragraphs/', TOPIC_SELECTION=TOPIC_SELECTION,
                LEMMATIZED_TEXT=paragraphs, DICTIONARY=dictionary, CORPUS=corpus, MIN_DF=MIN_DF, MAX_DF=MAX_DF, N_ITERATIONS=1000,
                PATH_TO_MALLET=r'C:/mallet/bin/mallet.bat', GET_COHERENCE_SCORE=True, COHERENCE='c_v')

## Load Multiple LDA Models